In [1]:
from IPython.display import display, Math, Latex

# Evaluation concept

We propose a framework for evaluating the performance of the FineScore, with a particular focus on its ability to capture nuanced changes in a patient's condition. The primary objective of the FineScore is to provide a more granular tracking of patient state transitions, including the detection of progression or regression, even in instances where the HB (House-Brackmann) score remains static.

For instance, consider a patient undergoing four sessions where the HB classification results are recorded as [4, 4, 4, 2]. This sequence indicates that the patient remains in the "Moderately Severe Dysfunction" category (HB grade 4) for three sessions, followed by an improvement to HB grade 2. In contrast, the FineScore for the same sessions might be represented as [4.8, 4.4, 3.8, 2.4], capturing incremental changes that are insufficient to shift the HB grade but still reflect subtle improvements in the patient's condition.

To quantify this phenomenon, both the HB score and the FineScore sequences are converted into a directional vector. This vector represents monotonic directional trends, where each segment is assigned a value of 1 (indicating an upward trend including previous potential stagnation), -1 (indicating a downward trend including previous potential stagnation), or 0 (indicating indeterminate direction due to no changes in HB grades). In the given example, both sequences exhibit a monotonic decline, resulting in the directional vector [-1, -1, -1]. Note that the directional vector contains one fewer element than the original sequence, as it measures directionality between consecutive points.

The next step involves assessing the alignment between the dynamics captured by the FineScore and the ground truth dynamics. To achieve this, we introduce the metric Mean Direction Match (1). The motivation behind the Mean Direction Match (MDM) lies in its ability to penalize discrepancies in sections where the directional dynamics of the FineScore do not align with the ground truth. This provides a clear and systematic approach to evaluating the FineScore's capacity to accurately detect directional changes in a patient's condition.

In [16]:
display(Math(r'mDM(k,l) = 1/N \sum_{i=1}^{N}k_i * l_i   \quad \quad   (1)'))

<IPython.core.display.Math object>

*where k and l are two directional vectors, N is the size of them*

The resulting values of the Mean Direction Match (MDM) fall within the range [-1, 1]. A value of -1 indicates a complete mismatch between the directional dynamics of the FineScore and the ground truth, while a value of 1 signifies a perfect alignment of directions. Values near 0 reflect instances of stagnation, where there is insufficient directional change to establish a clear trend. This range enables a nuanced interpretation of the FineScore's performance in capturing the trajectory of a patient's condition.

# Implementation

In [3]:
def to_directional_vector(seq):
    result = []
    direction = 0
    delay = 1
    for i in range(1, len(seq)):
        if seq[i] > seq[i-1]:
            direction = 1
            result += [direction] * delay
            delay = 1
        elif seq[i] < seq[i-1]:
            direction = -1
            result += [direction] * delay
            delay = 1
        else: 
            direction = 0
            delay += 1
    if delay > 1:    
        result += [direction] * (delay-1)        
    
    
    return result

In [4]:
def mean_direction_match(xs, ys):
    assert len(xs) == len(ys)
    total = [(x * y) for x,y in zip(xs, ys)]

    return sum(total) / len(xs)

In [5]:
hb = [4, 4, 3, 2]
fs = [4.8, 4.4, 3.8, 2.4]
mDM = mean_direction_match(
    to_directional_vector(hb),
    to_directional_vector(fs)
)
print(f'HB {hb}')
print(f'FS {fs}')
print(f'mean Directional Match {mDM}')

HB [4, 4, 3, 2]
FS [4.8, 4.4, 3.8, 2.4]
mean Directional Match 1.0


# Tests

In [6]:
stable = ([1,1,1,1], [0, 0, 0])
mono_up = ([1,1,3,6], [1, 1, 1])
mono_down = ([4,4,3,2], [-1, -1, -1])
change = ([1,2,6,3], [1, 1, -1])
change_2 = ([1,4,2,3], [1, -1, 1])
change_stable = ([1,4,3,3], [1, -1, 0])

In [7]:
fine_score = ([4.8, 4.4, 3.8, 2.4], [-1, -1, -1])

## Test direction vector

In [9]:
test_cases = [stable, mono_up, mono_down, change, change_2, change_stable, fine_score]

for test_case in test_cases:
    res = to_directional_vector(test_case[0])
    print('testing:')
    print(f'   input:  {test_case[0]}')
    print(f'   output: {res}')
    assert res == test_case[1]
    print(f'   {test_case[1] == res}')

testing:
   input:  [1, 1, 1, 1]
   output: [0, 0, 0]
   True
testing:
   input:  [1, 1, 3, 6]
   output: [1, 1, 1]
   True
testing:
   input:  [4, 4, 3, 2]
   output: [-1, -1, -1]
   True
testing:
   input:  [1, 2, 6, 3]
   output: [1, 1, -1]
   True
testing:
   input:  [1, 4, 2, 3]
   output: [1, -1, 1]
   True
testing:
   input:  [1, 4, 3, 3]
   output: [1, -1, 0]
   True
testing:
   input:  [4.8, 4.4, 3.8, 2.4]
   output: [-1, -1, -1]
   True


**END**